In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


In [21]:
print(os.listdir("/gdrive/My Drive/Banana_Ripeness_Classification"))

['unripe', 'ripe', 'rotten', 'overripe']


In [22]:
# Veri setini yükleme ve ön işleme
DATADIR = '/gdrive/My Drive/Banana_Ripeness_Classification'
CATEGORIES = ["overripe", "ripe", "rotten", "unripe"]
IMG_SIZE = 100

In [23]:
all_data = []
def create_all_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                all_data.append([new_array, class_num])
            except Exception as e:
                pass

create_all_data()

In [24]:
print("toplam görüntü sayısı: ",len(all_data))

toplam görüntü sayısı:  13478


In [25]:
X = []
y = []
for features, label in all_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE * IMG_SIZE * 3) # Düzleştirme işlemi
y = np.array(y)

In [26]:
# Veriyi bölme
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)


In [27]:
# Etiketleri one-hot kodlama
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)
y_val = to_categorical(y_val, num_classes=4)


In [28]:
from keras.models import Sequential
from keras.layers import Dense

In [29]:
# Modeli oluşturma
model = Sequential()
model.add(Dense(1024, input_shape=(IMG_SIZE * IMG_SIZE * 3,), activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(4, activation='softmax')) # Çıkış katmanı 4 nöronlu olmalı ve 'softmax' kullanmalı


In [30]:
# Modeli derleme
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])


In [33]:
# Modeli eğitme
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))


Epoch 1/10
266/266 [==============================] - 211s 792ms/step - loss: 33.6644 - accuracy: 0.5058 - val_loss: 41.3056 - val_accuracy: 0.5021
Epoch 2/10
266/266 [==============================] - 207s 778ms/step - loss: 19.8143 - accuracy: 0.5102 - val_loss: 13.6941 - val_accuracy: 0.5275
Epoch 3/10
266/266 [==============================] - 209s 785ms/step - loss: 7.0698 - accuracy: 0.5515 - val_loss: 14.0578 - val_accuracy: 0.3379
Epoch 4/10
266/266 [==============================] - 203s 763ms/step - loss: 4.3170 - accuracy: 0.5490 - val_loss: 3.5881 - val_accuracy: 0.4936
Epoch 5/10
266/266 [==============================] - 209s 786ms/step - loss: 2.2632 - accuracy: 0.5912 - val_loss: 2.2871 - val_accuracy: 0.4778
Epoch 6/10
266/266 [==============================] - 204s 768ms/step - loss: 1.6898 - accuracy: 0.6226 - val_loss: 2.7517 - val_accuracy: 0.4725
Epoch 7/10
266/266 [==============================] - 207s 780ms/step - loss: 1.7524 - accuracy: 0.6097 - val_loss: 1.1

In [34]:
# Test seti üzerinde değerlendirme
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)

127/127 [==============================] - 15s 118ms/step


In [35]:
# Performans metrikleri
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
# Sonuçları gösterme
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))

Accuracy:  0.3365479723046489
Precision:  0.27491133810391855
Recall:  0.3365479723046489
F1 Score:  0.17381507566626353

Confusion Matrix:
 [[   0    1  821    0]
 [   0   11 1182    0]
 [   0    8 1350    0]
 [   0    0  671    0]]

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       822
           1       0.55      0.01      0.02      1193
           2       0.34      0.99      0.50      1358
           3       0.00      0.00      0.00       671

    accuracy                           0.34      4044
   macro avg       0.22      0.25      0.13      4044
weighted avg       0.27      0.34      0.17      4044



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
